In [2]:
import numpy as np
import pandas as pd
import scipy.stats as st
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
np.random.seed(42)

df = pd.DataFrame({
    'value': [ 
        1.26,   2.78,   0.63,   0.34,
        3.53,   4.10,   1.31,   0.91,
        11.98,  13.14,  9.86,   6.53,
        90.82,  97.12,  75.85,  59.46,
        83.45,  116.62, 104.00, 80.85,
        55.98,  67.28,  79.33,  82.66,
        66.32,  78.53,  69.10,  67.27,
        37.42,  55.35,  60.76,  73.20
    ], 
    'lambda': [ 
        2,    2,    2,    2,   
        7,    7,    7,    7,    
        12,   12,   12,   12,   
        19.5, 19.5, 19.5, 19.5, 
        29.5, 29.5, 29.5, 29.5, 
        39.5, 39.5, 39.5, 39.5, 
        54.5, 54.5, 54.5, 54.5, 
        75,   75,   75,   75    
    ]
})

xs = [float(i) for i in df['lambda'].unique()]
ys = [np.mean(df[ df['lambda'].values == x_i ].value.values) for x_i in xs]
data = [df[ df['lambda'].values == x_i ].value.values for x_i in xs]

In [11]:
def isiterable(obj):
    try:
        it = iter(obj)
    except TypeError: 
        return False
    return True

def eval(func, vars):
    if isiterable(vars):
        return [ func(var) for var in vars ]
    return func(vars)

def replace(vec, indize, value):
    if isiterable(indize):
        for i in range(len(indize)):
            vec[ indize[i] ] = value[i]
    else:
        vec[ indize ] = value  
    return vec      

def derivative(func, vars, n=1, h=0.0001, method='central'): 
    if method == 'central':
        f = lambda var: (func(var + h) - func(var - h))/(2*h)
    elif method == 'forward':
        f = lambda var: (func(var + h) - func(var))/(h)
    elif method == 'backward':
        f = lambda var: (func(var) - func(var - h))/(h)
    if n>1: 
        deriv = lambda x: derivative(f, x, n-1, h, method) 
        return eval(deriv, vars)
    return eval(f, vars)

def Derivative(func, vec, h=0.0001):
    n = len(vec)
    D = np.zeros(n)
    for i in range(n):
        d1 = func( replace(vec, i, vec[i]+h) )
        d2 = func( replace(vec, i, vec[i]-h) )
        d = d1/(2*h) - d2/(2*h) #( d1-d2 )/(2*h)
        D[i] = d
    return D 

def Hessian(func, vec, h=0.0001):
    n = len(vec)
    H = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            d1 = func( replace(vec, [i,j], [vec[i]+h,vec[j]+h]) )
            d2 = func( replace(vec, [i,j], [vec[i]+h,vec[j]-h]) )
            d3 = func( replace(vec, [i,j], [vec[i]-h,vec[j]+h]) )
            d4 = func( replace(vec, [i,j], [vec[i]-h,vec[j]-h]) )
            d =  d1/(4*h**2) - d2/(4*h**2) - d3/(4*h**2) + d4/(4*h**2) #( (d1-d2)/(2*h) - (d3-d4)/(2*h) )/(2*h)
            H[i][j] = d
    return H

def eta(x, theta=[153.79, 17.26, -2.58, 0.455, 0.01746]): 
    f = lambda var: (theta[0] + theta[2]*var + theta[4]*var**2) * np.exp(theta[3]*(var-theta[1]))/( 1 + np.exp(theta[3]*(var-theta[1])) )
    return eval(f, x)

def y(x, theta=[153.79, 17.26, -2.58, 0.455, 0.01746], sigma=10.17): 
    f = lambda var: eta(var, theta) + np.random.normal(0, sigma**2)
    return eval(f, x)

def f(i, y, theta=[153.79, 17.26, -2.58, 0.455, 0.01746], sigma=10.17, x=xs):
    f = lambda var: np.exp( -(var - eta(x[i], theta))**2 / (2*sigma**2) ) / np.sqrt(2*np.pi*sigma**2)
    return eval(f, y)

def g(theta, x=xs):
    return eta(x, theta)

def Lik(theta, y=ys):
    return np.prod( [f(i, y[i], theta) for i in range(len(y))] )

def L(theta, y=ys):
    return np.log( Lik(theta, y) )

def L_sum(theta, y=ys):
    return np.sum( [np.log(f(i, y[i], theta)) for i in range(len(y))] ) 

# a=10.17, b=153.79, c=17.26, d=-2.58, e=0.455, f=0.01746 
print(Lik([153.79, 17.26, -2.58, 0.455, 0.01746]))
print(L([153.79, 17.26, -2.58, 0.455, 0.01746]))
print(L_sum([153.79, 17.26, -2.58, 0.455, 0.01746]))
print(Derivative(L, [153.79, 17.26, -2.58, 0.455, 0.01746]))
myH = Hessian(L_sum, [153.79, 17.26, -2.58, 0.455, 0.01746])
V = np.linalg.inv(-myH)
test = Derivative(eta)

# x = np.linspace(-10, 10, num=100)
# test = lambda x: x**5
# plt.plot(x, derivative(test, x, 3))

3.320077477658087e-12
-26.431032996628446
-26.431032996628446
[-3.58636049e-03  8.28299642e-04 -2.46751188e-01 -9.92568326e-03
 -2.79499184e+01]


In [ ]:
# Likelyhood functions

In [ ]:
# Random MLE
max = 0
param = []
for i in range(100000):
    a = np.random.uniform(150, 160)
    b = np.random.uniform(15, 20)
    c = np.random.uniform(-5, 0)
    d = np.random.uniform(-2, 2)
    e = np.random.uniform(-2, 2)
    likelyHood = Lik([a, b, c, d, e], ys)
    if likelyHood>max: 
        max=likelyHood
        param = [a,b,c,d,e]
        dvt = derivate_L(param)
print(max)
print(param)
print(dvt)

In [ ]:
# Data
plt.plot(xs, data, 'o', color='black', markersize='3')

# Regression
x = np.linspace(1, 80, num=100)
plt.plot(x, eta(x))

# TODO Confidence Bands
# plt.plot(x, [eta(x_i) - h_lower(x_i) for x_i in x], linestyle='dashed')
# plt.plot(x, [eta(x_i) + h_upper(x_i) for x_i in x], linestyle='dashed')